In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [0]:
# Данный код монтирует гугл диск к директории /gdrive
from google.colab import drive
drive.mount('/gdrive')

In [0]:
orange_data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/FINAL_2/DATA/orange_small_churn_data.train')
orange_data.shape

(40000, 230)

In [0]:
orange_labels = pd.read_csv('/gdrive/My Drive/Colab Notebooks/FINAL_2/DATA/orange_small_churn_labels.train', header=None, names=['label'])

In [0]:
train = pd.concat([orange_data, orange_labels], axis=1)
train.shape

(40000, 231)

**Так как классы не сбалансированы проводим oversampling, добавляем необходимое количество строк с классом "отток" так, чтобы была пропорция классов примерно 1:1**

In [0]:
from sklearn.utils import shuffle

def oversampling(data):
    dataChurn = data[data['label'] == 1]
    dataNoChurn = data[data['label'] == -1]
    alpha = (int)(dataNoChurn.shape[0]/dataChurn.shape[0])
    dataChurnRet = dataChurn 
    for i in range(alpha):
        dataChurnRet = dataChurnRet.append(dataChurn)
    dataNoChurn = dataNoChurn.append(dataChurnRet)
    dataNoChurn = shuffle(dataNoChurn)
    return dataNoChurn.iloc[:,:230], dataNoChurn.iloc[:,230:]

**Заменяем все пропуски в датасете на нули:**

In [0]:
train_data = train.fillna(0)

**Проводим oversampling для того, чтобы сделать пропорцию классов 1:1**

In [0]:
x_data, y_data = oversampling(train_data)

**Для преобразования категориальных фич был использован preprocessing.LabelEncoder**

In [0]:
from sklearn.preprocessing import LabelEncoder

def labelEncoder(data):
    data = data.fillna(0)
    le = LabelEncoder()
    for m in range(190,230):
        #print(m)
        tmp = data.iloc[:,m].astype(str)
        le.fit(tmp)
        data.iloc[:,m] = le.transform(tmp)        
    return data

In [0]:
x_data_le = labelEncoder(x_data)

**Создам функцию для разделения и оценки**

In [0]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score, precision_recall_curve, average_precision_score, roc_curve, auc

**RandomForestClassifier**

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
test = pd.read_csv('/gdrive/My Drive/Colab Notebooks/FINAL_2/DATA/orange_small_churn_test_data.csv')


In [0]:
test_data = test.fillna(0)

In [0]:
test_data.shape

(10000, 231)

In [0]:
def labelEncoder(data):
    data = data.fillna(0)
    le = LabelEncoder()
    for m in range(191,231):
        #print(m)
        tmp = data.iloc[:,m].astype(str)
        le.fit(tmp)
        data.iloc[:,m] = le.transform(tmp)        
    return data

In [0]:
x_test_data_le = labelEncoder(test_data)

In [0]:
clf=RandomForestClassifier(n_estimators=100)

In [0]:
clf.fit(x_data_le, y_data.values.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
pred = clf.predict(x_test_data_le.iloc[:, 1:])

In [0]:
idx = x_test_data_le.iloc[:, 0]

In [0]:
df = pd.DataFrame({'ID': idx.values, 'result': pred})
df.to_csv('answer1.csv', index = False)

In [0]:
from google.colab import files
files.download('answer1.csv')

**В результате получил только 0,5000**

In [0]:
#попробую еще раз деревья, но без семплинга
clf.fit(x_data_le, y_data.values.ravel())

In [0]:
def labelEncoder(data):
    data = data.fillna(0)
    le = LabelEncoder()
    for m in range(190,230):
        #print(m)
        tmp = data.iloc[:,m].astype(str)
        le.fit(tmp)
        data.iloc[:,m] = le.transform(tmp)        
    return data

In [0]:
x_data_le = labelEncoder(train_data.iloc[:,:230])

In [0]:
clf.fit(x_data_le, train_data['label'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
pred = clf.predict(x_test_data_le.iloc[:, 1:])
idx = x_test_data_le.iloc[:, 0]
df = pd.DataFra0.50137me({'ID': idx.values, 'result': pred})
df.to_csv('answer2.csv', index = False)
from google.colab import files
files.download('answer2.csv')

**не изменилось - те же 0,5000**

In [0]:
# проведем испытания XGBoost
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from scipy.stats import uniform, randint
from xgboost import XGBClassifier
import time

In [0]:
x_train = x_data_le
y_train = train_data['label']

In [0]:
clf = XGBClassifier()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}

In [0]:
rs_clf = RandomizedSearchCV(clf, param_grid, n_iter=20,
                            n_jobs=1, verbose=2, cv=2,
                            scoring='neg_log_loss', refit=False, random_state=42)
print("Randomized search..")
search_time_start = time.time()
rs_clf.fit(x_train, y_train)
print("Randomized search time:", time.time() - search_time_start)

best_score = rs_clf.best_score_
best_params = rs_clf.best_params_
print("Best score: {}".format(best_score))
print("Best params: ")
for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))

Randomized search..
Fitting 2 folds for each of 20 candidates, totalling 40 fits
[CV] subsample=0.7, silent=False, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  subsample=0.7, silent=False, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4, total=  12.2s
[CV] subsample=0.7, silent=False, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.2s remaining:    0.0s


[CV]  subsample=0.7, silent=False, reg_lambda=50.0, n_estimators=100, min_child_weight=5.0, max_depth=6, learning_rate=0.2, gamma=1.0, colsample_bytree=0.9, colsample_bylevel=0.4, total=  12.1s
[CV] subsample=0.6, silent=False, reg_lambda=1.0, n_estimators=100, min_child_weight=1.0, max_depth=20, learning_rate=0.1, gamma=0.25, colsample_bytree=0.8, colsample_bylevel=0.8 
[CV]  subsample=0.6, silent=False, reg_lambda=1.0, n_estimators=100, min_child_weight=1.0, max_depth=20, learning_rate=0.1, gamma=0.25, colsample_bytree=0.8, colsample_bylevel=0.8, total=  54.4s
[CV] subsample=0.6, silent=False, reg_lambda=1.0, n_estimators=100, min_child_weight=1.0, max_depth=20, learning_rate=0.1, gamma=0.25, colsample_bytree=0.8, colsample_bylevel=0.8 
[CV]  subsample=0.6, silent=False, reg_lambda=1.0, n_estimators=100, min_child_weight=1.0, max_depth=20, learning_rate=0.1, gamma=0.25, colsample_bytree=0.8, colsample_bylevel=0.8, total=  53.8s
[CV] subsample=0.9, silent=False, reg_lambda=50.0, n_est

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[CV]  subsample=0.6, silent=False, reg_lambda=100.0, n_estimators=100, min_child_weight=7.0, max_depth=10, learning_rate=3, gamma=0, colsample_bytree=0.6, colsample_bylevel=0.5, total=   9.2s
[CV] subsample=0.6, silent=False, reg_lambda=100.0, n_estimators=100, min_child_weight=7.0, max_depth=20, learning_rate=0.2, gamma=0.25, colsample_bytree=1.0, colsample_bylevel=0.5 
[CV]  subsample=0.6, silent=False, reg_lambda=100.0, n_estimators=100, min_child_weight=7.0, max_depth=20, learning_rate=0.2, gamma=0.25, colsample_bytree=1.0, colsample_bylevel=0.5, total=  25.6s
[CV] subsample=0.6, silent=False, reg_lambda=100.0, n_estimators=100, min_child_weight=7.0, max_depth=20, learning_rate=0.2, gamma=0.25, colsample_bytree=1.0, colsample_bylevel=0.5 
[CV]  subsample=0.6, silent=False, reg_lambda=100.0, n_estimators=100, min_child_weight=7.0, max_depth=20, learning_rate=0.2, gamma=0.25, colsample_bytree=1.0, colsample_bylevel=0.5, total=  26.1s
[CV] subsample=0.7, silent=False, reg_lambda=5.0, 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[CV]  subsample=0.7, silent=False, reg_lambda=5.0, n_estimators=100, min_child_weight=1.0, max_depth=15, learning_rate=3, gamma=1.0, colsample_bytree=0.5, colsample_bylevel=0.4, total=   7.8s
[CV] subsample=0.7, silent=False, reg_lambda=5.0, n_estimators=100, min_child_weight=1.0, max_depth=15, learning_rate=3, gamma=1.0, colsample_bytree=0.5, colsample_bylevel=0.4 
[CV]  subsample=0.7, silent=False, reg_lambda=5.0, n_estimators=100, min_child_weight=1.0, max_depth=15, learning_rate=3, gamma=1.0, colsample_bytree=0.5, colsample_bylevel=0.4, total=   6.2s
Randomized search time: 867.0530691146851
Best score: -0.2406829303286737
Best params: 
colsample_bylevel: 0.4
colsample_bytree: 1.0
gamma: 0.5
learning_rate: 0.1
max_depth: 20
min_child_weight: 7.0
n_estimators: 100
reg_lambda: 100.0
silent: False
subsample: 0.8


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 14.5min finished


In [0]:
clf_best = XGBClassifier(colsample_bylevel=0.4, colsample_bytree=1.0, gamma = 0.5, learning_rate=0.1, max_depth=20, min_child_weight=7.0, n_estimators=100,
                         reg_lambda=100.0, silent=False, subsample=0.8)

In [0]:
clf_best.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.4,
              colsample_bynode=1, colsample_bytree=1.0, gamma=0.5,
              learning_rate=0.1, max_delta_step=0, max_depth=20,
              min_child_weight=7.0, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=100.0, scale_pos_weight=1, seed=None,
              silent=False, subsample=0.8, verbosity=1)

In [0]:
pred = clf_best.predict(x_test_data_le.iloc[:, 1:])

In [0]:
idx = x_test_data_le.iloc[:, 0]
df = pd.DataFrame({'ID': idx.values, 'result': pred})
df.to_csv('answer3.csv', index = False)
from google.colab import files
files.download('answer3.csv')

**0.50137**